# Coarse to fine

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import ParameterSampler
%matplotlib inline
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Utils.createURM import createURM

In [2]:
URM = createURM()

/Users/matteopancini/PycharmProjects/recsys-challenge-2022-Pancini-Vitali/Utils/createURM.py:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('../Input/interactions_and_impressions.csv')


In [3]:
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.80)

In [4]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 726 ( 1.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 329 ( 0.8%) Users that have less than 1 test interactions


In [5]:
grid_size = 100
TUNE_ITER = 100
num_epochs = 5
worse_score = 0

init_param_grid = {'topK': [i for i in range(10, 500)],
                   'shrink': [i for i in range(10, 100)],
                  }

new_param_grid = init_param_grid.copy()
best_params_dict = {'score':worse_score,'params':[]}
tried_params_list = []

for epoch in range(num_epochs):

    # List of sampled hyperparameter combinations will be used for random search
    param_list = list(ParameterSampler(new_param_grid, n_iter=TUNE_ITER,random_state=0))

    # Searching the Best Parameters with Random Search
    rs_results_dict = {}
    for tune_iter in range(TUNE_ITER):
        # Get the set of parameter for this iteration
        strategy_params = param_list[tune_iter]

        recommender = ItemKNNCFRecommender(URM_train)
        recommender.fit(shrink=strategy_params['shrink'], topK=strategy_params['topK'])
        results, _ = evaluator_validation.evaluateRecommender(recommender)
        results = results.loc[10]['MAP']

        rs_results_dict[tuple(strategy_params.values())] = {'score':results}

        if results > best_params_dict['score']:
            best_params_dict['score'] = results
            best_params_dict['params'] = list(strategy_params.values())

    # Save the results in dataframe and sort it based on score
    df_rs_results = pd.DataFrame(rs_results_dict).T.reset_index()
    df_rs_results.columns = list(strategy_params.keys()) + ['score']
    df_rs_results = df_rs_results.sort_values(['score'],ascending=False).head(num_epochs-epoch)

    # If the best score from this epoch is worse than the best score,
    # then append the best hyperaparameters combination to this epoch dataframe
    if df_rs_results['score'].iloc[0] < best_params_dict['score']:
        new_row_dict = {}
        new_row_dict['score'] = best_params_dict['score']
        for idx, key in enumerate(init_param_grid):
            new_row_dict[key] = best_params_dict['params'][idx]

        df_rs_results = df_rs_results.append(new_row_dict,ignore_index=True)
        df_rs_results = df_rs_results.sort_values(['score'],ascending=False).head(num_epochs-epoch)

    display(df_rs_results)
    print(df_rs_results.head(1).T.to_dict())

    # Get the worse and best hyperparameter combinations
    df_rs_results_min = df_rs_results[df_rs_results['score']>worse_score].min(axis=0)
    df_rs_results_max = df_rs_results[df_rs_results['score']>worse_score].max(axis=0)

    # Generate new hyperparameter space based on current worse and best hyperparameter combinations
    for key in init_param_grid:
        if isinstance(init_param_grid[key][0],int):
            new_param_grid[key] = np.unique([i for i in range(int(df_rs_results_min[key]), int(df_rs_results_max[key])+1)])
        elif isinstance(init_param_grid[key][0],float):
            new_param_grid[key] = np.unique(np.linspace(df_rs_results_min[key], df_rs_results_max[key], grid_size))
        else:
            new_param_grid[key] = init_param_grid[key]

    # Decrease the tuning iteration for random search
    TUNE_ITER = int(TUNE_ITER - epoch * TUNE_ITER/num_epochs)

ItemKNNCFRecommender: URM Detected 1 ( 0.0%) users with no interactions.
Similarity column 24507 (100.0%), 8499.92 column/sec. Elapsed time 2.88 sec
EvaluatorHoldout: Processed 40903 (100.0%) in 25.98 sec. Users per second: 1574
ItemKNNCFRecommender: URM Detected 1 ( 0.0%) users with no interactions.
Similarity column 24507 (100.0%), 8614.82 column/sec. Elapsed time 2.84 sec
EvaluatorHoldout: Processed 40903 (100.0%) in 29.40 sec. Users per second: 1391
ItemKNNCFRecommender: URM Detected 1 ( 0.0%) users with no interactions.
Similarity column 24507 (100.0%), 8704.95 column/sec. Elapsed time 2.82 sec
EvaluatorHoldout: Processed 40903 (100.0%) in 29.66 sec. Users per second: 1379
ItemKNNCFRecommender: URM Detected 1 ( 0.0%) users with no interactions.
Similarity column 24507 (100.0%), 8715.65 column/sec. Elapsed time 2.81 sec
EvaluatorHoldout: Processed 40903 (100.0%) in 21.28 sec. Users per second: 1922
ItemKNNCFRecommender: URM Detected 1 ( 0.0%) users with no interactions.
Similarity 

,topK,shrink,score
3,183,53,0.019323
7,187,52,0.019318


{3: {'topK': 183.0, 'shrink': 53.0, 'score': 0.019322663878808606}}
ItemKNNCFRecommender: URM Detected 1 ( 0.0%) users with no interactions.
Similarity column 24507 (100.0%), 8670.02 column/sec. Elapsed time 2.83 sec
EvaluatorHoldout: Processed 40903 (100.0%) in 21.39 sec. Users per second: 1912
ItemKNNCFRecommender: URM Detected 1 ( 0.0%) users with no interactions.
Similarity column 24507 (100.0%), 8561.19 column/sec. Elapsed time 2.86 sec
EvaluatorHoldout: Processed 40903 (100.0%) in 21.84 sec. Users per second: 1873
ItemKNNCFRecommender: URM Detected 1 ( 0.0%) users with no interactions.
Similarity column 24507 (100.0%), 8515.81 column/sec. Elapsed time 2.88 sec
EvaluatorHoldout: Processed 40903 (100.0%) in 21.71 sec. Users per second: 1884
ItemKNNCFRecommender: URM Detected 1 ( 0.0%) users with no interactions.
Similarity column 24507 (100.0%), 8736.87 column/sec. Elapsed time 2.81 sec
EvaluatorHoldout: Processed 40903 (100.0%) in 21.58 sec. Users per second: 1895
ItemKNNCFRecomme

,topK,shrink,score
3,186,52,0.019324


{3: {'topK': 186.0, 'shrink': 52.0, 'score': 0.019324115241284825}}


In [37]:
import json

importantParameters = df_rs_results.to_json(orient="records")
parsed = json.loads(importantParameters)

with open("logs/" + recommender.RECOMMENDER_NAME + "_logs.json", 'w') as json_file:
        json.dump(parsed, json_file, indent=4)